In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
activity = pd.read_csv('data/features/activity.csv')
performances = pd.read_csv('data/features/performances.csv')

In [3]:
performances_math = performances[performances['domain']== 'math'].copy()
activity_math = activity[activity['domain']== 'math'].copy()

In [4]:
len(activity_math)

25938

In [5]:
activity_math.isnull().sum()

activity_id               0
user_id                   0
post_id                   0
course_id                 0
activity_type             0
activity_status           0
activity_started          0
activity_completed    12676
activity_updated          0
domain                    0
date_restored             0
times_valid               0
date                      0
time_spent                0
time_in_minutes           0
time_truncated            0
dtype: int64

In [6]:
activity_math.dropna(inplace=True)

We can see that almost 50% of the rows contain nans in the activity_completed column. Unfortunately we have to drop all these rows for our analysis because 2 out of our 7 features depend on the time spent on the activities which we cannot compute without the activity completed value. (we can see in the "Annex : Test without the time dependant activity features" section that keeping all the data and removing those 2 time dependant features makes our model worst which confirms our choice)

### Feature extraction

In [7]:
# Rolling window for recent activity
rolling_window_days = 10

# Convert the 'date' columns to datetime
activity_math['activity_updated'] = pd.to_datetime(activity_math['activity_updated'])
performances_math['time'] = pd.to_datetime(performances_math['time'])

def compute_all_features_for_exam(exam_row, user_activities, user_exams, window_days=rolling_window_days):

    exam_dt = exam_row['time']

    # Include activities up to and including exam_date
    previous_activities = user_activities[user_activities['activity_updated'] < exam_dt].copy()

    # Rolling window (activities in the last N days, including exam day)
    window_start = exam_dt - pd.Timedelta(days=window_days)
    rolling_activities = previous_activities[previous_activities['activity_updated'] >= window_start].copy()

    features = {}

    # Recent average time per activity (rolling window)
    total_time_rolling = rolling_activities['time_in_minutes'].sum()
    count_rolling = len(rolling_activities)
    features['recent_avg_time_per_activity'] = total_time_rolling / count_rolling if count_rolling > 0 else 0

    # Number of days since last activity
    if not previous_activities.empty:
        last_activity_date = previous_activities['activity_updated'].max()
        features['days_since_last_activity'] = (exam_dt - last_activity_date).days
    else:
        features['days_since_last_activity'] = np.nan

    # Total time spent on activities before the exam
    features['total_time_spent_on_activity_before_exam'] = previous_activities['time_in_minutes'].sum() if not previous_activities.empty else 0

    # Average percentage on past exams
    previous_exams = user_exams[user_exams['time'] < exam_dt]
    features['average_performance_past_exams'] = previous_exams['performance'].mean() if not previous_exams.empty else np.nan

    # Usage Frequency: Average activities per day in rolling window & Active days ratio
    features['avg_activities_per_day_recent'] = count_rolling / window_days if window_days > 0 else np.nan
    if not rolling_activities.empty:
        distinct_days = rolling_activities['activity_updated'].dt.normalize().nunique()
    else:
        distinct_days = 0
    features['active_days_ratio_recent'] = distinct_days / window_days if window_days > 0 else np.nan

    # Activity diversity (rolling window)
    features['diversity_recent'] = rolling_activities['activity_type'].nunique() if not rolling_activities.empty else 0


    return pd.Series(features)

# Loop over each exam (grouped by user) in performances_math and compute all features.
features_list = []

for user_id, user_exams in performances_math.groupby('user_id'):
    # Get corresponding activities for the user from activity_math and sort by date
    user_activities = activity_math[activity_math['user_id'] == user_id].sort_values('activity_updated')
    user_exams_sorted = user_exams.sort_values('time')

    for exam_index, exam_row in user_exams_sorted.iterrows():
        feats = compute_all_features_for_exam(exam_row, user_activities, user_exams_sorted, rolling_window_days)
        feats['exam_index'] = exam_index
        features_list.append(feats)

# Output df
features_df = pd.DataFrame(features_list).set_index('exam_index')
performances_math_features = performances_math.join(features_df, how='left')

In [8]:
performances_math_features.isnull().sum()

user_id                                       0
domain                                        0
test_id                                       0
course                                        0
date                                          0
time                                          0
percentage                                    0
performance                                   0
recent_avg_time_per_activity                  0
days_since_last_activity                     56
total_time_spent_on_activity_before_exam      0
average_performance_past_exams              469
avg_activities_per_day_recent                 0
active_days_ratio_recent                      0
diversity_recent                              0
dtype: int64

We can see that we have quite a lot of Nans in the average_percentage_past_exams column, Those correspond to the first exam questions that every user did for which there is no past value. We will first try to train a linear model by dropping those rows and then we will try to impute these values with the average exam percentage of similar users and see which model performs best.

### Model where we just drop all rows containing Nans

In [9]:
performances_math_features_drop = performances_math_features.dropna()

In [10]:
from sklearn.preprocessing import StandardScaler
# scaling the columns
columns_to_scale = ['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam','average_performance_past_exams','avg_activities_per_day_recent','active_days_ratio_recent','diversity_recent']


scaler = StandardScaler()
scaled_values = scaler.fit_transform(performances_math_features_drop[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performances_math_features_drop.index)
remaining_df = performances_math_features_drop.drop(columns=columns_to_scale)
final_df_drop = pd.concat([scaled_df, remaining_df], axis=1)

In [11]:
import statsmodels.formula.api as smf

# Linear Regression Model
mod = smf.ols(formula= 'performance ~  recent_avg_time_per_activity + days_since_last_activity + total_time_spent_on_activity_before_exam + average_performance_past_exams + avg_activities_per_day_recent + active_days_ratio_recent + diversity_recent', data=final_df_drop)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.267
Model:                            OLS   Adj. R-squared:                  0.266
Method:                 Least Squares   F-statistic:                     173.6
Date:                Thu, 24 Apr 2025   Prob (F-statistic):          1.25e-219
Time:                        20:05:53   Log-Likelihood:                -15421.
No. Observations:                3340   AIC:                         3.086e+04
Df Residuals:                    3332   BIC:                         3.091e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

We get an R^2 value of 0.267 and can see that the average percentage on last exam seems to be the most important feature out of the lot by quite a margin lets thus try to impute the missing average_percentage_past_exams values missing to see if we can improve our model. we also saw that using the average performance on last exams intead of the avergae percenatge on last exams improves the model quite a bit.... TO DEVELOPP MORE

### Now lets try to impute the misssing average_perfomance_last_exam_values

In [12]:
performance_math_impute = performances_math_features.copy()

In [13]:
from sklearn.neighbors import NearestNeighbors

# Features we’ll use to compute similarity:
sim_features = ['recent_avg_time_per_activity','days_since_last_activity','total_time_spent_on_activity_before_exam',
                'avg_activities_per_day_recent','active_days_ratio_recent','diversity_recent']

# New column to hold the imputed values
performance_math_impute['avg_perf_past_exams_imputed'] = performance_math_impute['average_performance_past_exams']

# Group by test_id and run kNN inside each group
for test_id, group in performance_math_impute.groupby('test_id'):
    # indices of rows we need to fill
    missing_idx = group[group['average_performance_past_exams'].isna()].index
    if len(missing_idx) == 0:
        continue

    # candidate neighbors: same test, non‐missing avg_pct
    candidates = group[group['average_performance_past_exams'].notna()]
    if candidates.shape[0] == 0:
        # if no one else took that test --> impute median
        continue

    # Matrix of sim_features, impute median for remaining NaNs
    feat_mat = group[sim_features].copy()
    feat_mat = feat_mat.fillna(feat_mat.median())

    # Split into X_train (candidates) and X_query (the missing rows)
    X_train = feat_mat.loc[candidates.index].values
    X_query = feat_mat.loc[missing_idx].values

    # Use up to 3 neighbors (fewer if not enough candidates)
    k = min(3, X_train.shape[0])
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(X_train)
    distances, neighbors = nbrs.kneighbors(X_query)

    # For each missing row, average the actual test scores of its neighbors
    candidate_scores = candidates['performance'].values
    for i, idx in enumerate(missing_idx):
        nbr_idxs = neighbors[i]
        imputed_val = candidate_scores[nbr_idxs].mean()
        performance_math_impute.at[idx, 'avg_perf_past_exams_imputed'] = imputed_val

# Replace the original column
performance_math_impute['average_performance_past_exams'] = performance_math_impute['avg_perf_past_exams_imputed']
performance_math_impute.drop(columns='avg_perf_past_exams_imputed', inplace=True)

In [14]:
# scaling the columns
columns_to_scale = ['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam','average_performance_past_exams','avg_activities_per_day_recent','active_days_ratio_recent','diversity_recent']
scaler = StandardScaler()
scaled_values = scaler.fit_transform(performance_math_impute[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performance_math_impute.index)
remaining_df = performance_math_impute.drop(columns=columns_to_scale)
final_df_impute = pd.concat([scaled_df, remaining_df], axis=1)

In [15]:
final_df_impute.dropna(inplace=True) # drop the remaining NaNs

In [16]:
import statsmodels.formula.api as smf

# Linear Regression Model
mod = smf.ols(formula= 'performance ~  recent_avg_time_per_activity + days_since_last_activity + total_time_spent_on_activity_before_exam + average_performance_past_exams + avg_activities_per_day_recent + active_days_ratio_recent + diversity_recent', data=final_df_impute)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.240
Model:                            OLS   Adj. R-squared:                  0.238
Method:                 Least Squares   F-statistic:                     168.8
Date:                Thu, 24 Apr 2025   Prob (F-statistic):          1.34e-217
Time:                        20:05:58   Log-Likelihood:                -17353.
No. Observations:                3754   AIC:                         3.472e+04
Df Residuals:                    3746   BIC:                         3.477e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

We can see that imputing the values for the first exam of each student actually make the model worst (R^2 of 0.240) --> it is better to just drop the rows.


### let's now see if simply impute with the median of that exam

In [17]:
performance_math_median = performances_math_features.copy()

In [19]:
# Fill by test_id median
performance_math_median['average_performance_past_exams'] = (
    performance_math_median
    .groupby('test_id')['average_performance_past_exams']
    .transform(lambda x: x.fillna(x.median()))
    .fillna(performance_math_median['average_performance_past_exams'].median())   # fallback to global median if a test has no median
)

In [20]:
from sklearn.preprocessing import StandardScaler
# scaling the columns
columns_to_scale = ['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam','average_performance_past_exams','avg_activities_per_day_recent','active_days_ratio_recent','diversity_recent']


scaler = StandardScaler()
scaled_values = scaler.fit_transform(performance_math_median[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performance_math_median.index)
remaining_df = performance_math_median.drop(columns=columns_to_scale)
final_df_med = pd.concat([scaled_df, remaining_df], axis=1)

In [21]:
import statsmodels.formula.api as smf

# Linear Regression Model
mod = smf.ols(formula= 'performance ~  recent_avg_time_per_activity + days_since_last_activity + total_time_spent_on_activity_before_exam + average_performance_past_exams + avg_activities_per_day_recent + active_days_ratio_recent + diversity_recent', data=final_df_med)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.242
Model:                            OLS   Adj. R-squared:                  0.240
Method:                 Least Squares   F-statistic:                     170.6
Date:                Thu, 24 Apr 2025   Prob (F-statistic):          9.72e-220
Time:                        20:06:24   Log-Likelihood:                -17349.
No. Observations:                3754   AIC:                         3.471e+04
Df Residuals:                    3746   BIC:                         3.476e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                               coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------

doesn't improve the model either...

### Let's now see if another model that supports Nans performs better than simple linear regression

In [39]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error


df = performances_math_features.copy()

feature_cols = ['recent_avg_time_per_activity','days_since_last_activity','total_time_spent_on_activity_before_exam','average_performance_past_exams',
    'avg_activities_per_day_recent','active_days_ratio_recent','diversity_recent']

X = df[feature_cols]
y = df['performance']

# Train-Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline: impute → scale → gradient boost
pipeline = Pipeline([
    ('imputer',   SimpleImputer(strategy='median')),
    ('scaler',    StandardScaler()),
    ('gbr',       GradientBoostingRegressor(random_state=42))
])

# Grid search for key hyperparameters
param_grid = {'gbr__n_estimators': [100, 200], 'gbr__learning_rate': [0.05, 0.1], 'gbr__max_depth':[3, 5], 'gbr__subsample':[0.8, 1.0]}
grid = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1)

# Fit the model
grid.fit(X_train, y_train)
print("Best params:", grid.best_params_)
best_model = grid.best_estimator_

# Evaluate on the test set
y_pred = best_model.predict(X_test)
print("Test R²:", r2_score(y_test, y_pred))
print("Test RMSE:", np.sqrt(mean_squared_error(y_test, y_pred)))

# Cross‑validated performance
cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='r2', n_jobs=-1)
print("5‑fold CV R²: %0.3f ± %0.3f" % (cv_scores.mean(), cv_scores.std()))


Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best params: {'gbr__learning_rate': 0.05, 'gbr__max_depth': 3, 'gbr__n_estimators': 100, 'gbr__subsample': 0.8}
Test R²: 0.2753500683990725
Test RMSE: 23.89582817994709
5‑fold CV R²: 0.217 ± 0.081


The original R^2 on the test set is better than our linear model but with the cross validation we can see that it is actually less good on average. NEED TO SEE OTHER EVALUATION CRITERIA

### Let's see if the model improves if we first cluster the students and then train a mixed effects model

In [59]:
from sklearn.cluster import KMeans
import statsmodels.formula.api as smf

# Collapse to one row per user for clustering
user_feats = final_df_drop.groupby('user_id')[['recent_avg_time_per_activity', 'days_since_last_activity', 'total_time_spent_on_activity_before_exam', 'average_performance_past_exams', 'avg_activities_per_day_recent', 'active_days_ratio_recent', 'diversity_recent']].mean().reset_index()

# Cluster into 3 groups
kmeans = KMeans(n_clusters=3, random_state=0)
user_feats['cluster'] = kmeans.fit_predict(user_feats.drop(columns='user_id'))

# Merge cluster label back into final_df
df2 = final_df_drop.merge(user_feats[['user_id','cluster']], on='user_id')

# 4) Fit a mixed‐effects model with random intercept per cluster
md = smf.mixedlm("performance ~ recent_avg_time_per_activity + days_since_last_activity + \
     total_time_spent_on_activity_before_exam + average_performance_past_exams + \
     avg_activities_per_day_recent + active_days_ratio_recent + diversity_recent",df2,groups="cluster")
mdf = md.fit(reml=False)
print(mdf.summary())

# In-sample predictions
df2["pred_cluster"] = mdf.predict(df2)

# Compute R² and RMSE
r2_cluster = r2_score(df2["performance"], df2["pred_cluster"])
rmse_cluster = mean_squared_error(df2["performance"], df2["pred_cluster"])

print("Cluster‐model  AIC:",  mdf.aic, "  BIC:", mdf.bic)
print(f"Cluster‐model  R² = {r2_cluster:.3f},  RMSE = {rmse_cluster:.3f}")

                       Mixed Linear Model Regression Results
Model:                      MixedLM         Dependent Variable:         performance
No. Observations:           3340            Method:                     ML         
No. Groups:                 3               Scale:                      590.8551   
Min. group size:            15              Log-Likelihood:             -15400.5039
Max. group size:            1806            Converged:                  No         
Mean group size:            1113.3                                                 
-----------------------------------------------------------------------------------
                                         Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------
Intercept                                 0.126    2.762  0.046 0.964 -5.287  5.539
recent_avg_time_per_activity              1.670    0.466  3.582 0.000  0.756  2.584
days_since_last

/opt/anaconda3/envs/ada_project/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/envs/ada_project/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/anaconda3/envs/ada_project/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/anaconda3/envs/ada_project/lib/python3.12/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/opt/anaconda3/envs/ada_project/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimi

Bet results with 3 clusters but doesn't actually improve the model compare to the simple linear model. NEED ALSO TO ANALYSE THE AIC AND BICs for each model + Maybe add some viz

## Annexe : Test without the time dependant activity features

In [35]:
activity = pd.read_csv('data/features/activity.csv')
performances = pd.read_csv('data/features/performances.csv')

In [36]:
performances_math = performances[performances['domain']== 'math'].copy()
activity_math = activity[activity['domain']== 'math'].copy()

In [37]:
activity_math

,activity_id,user_id,post_id,course_id,activity_type,activity_status,activity_started,activity_completed,activity_updated,domain,date_restored,times_valid,date,time_spent,time_in_minutes,time_truncated
0,1128,2533,42,42,course,0,2023-04-07 16:42:35,2023-04-07 17:35:15,2023-04-07 17:35:15,math,True,True,2023-04-07,0 days 00:52:40,52.666667,False
1,1129,2533,55,42,lesson,0,2023-04-07 16:42:35,NaN,2023-04-07 16:42:35,math,False,True,2023-04-07,0 days 00:00:00,0.000000,False
2,1130,2533,98,42,topic,1,2023-04-07 16:42:38,2023-04-07 16:43:58,2023-04-07 16:43:58,math,False,True,2023-04-07,0 days 00:01:20,1.333333,False
3,1131,2533,100,42,topic,1,2023-04-07 16:43:59,2023-04-07 16:46:13,2023-04-07 16:46:13,math,False,True,2023-04-07,0 days 00:02:14,2.233333,False
4,1132,2533,102,42,topic,1,2023-04-07 16:46:14,2023-04-07 16:46:27,2023-04-07 16:46:27,math,False,True,2023-04-07,0 days 00:00:13,0.216667,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50137,114742,955,647,42,topic,0,2025-03-07 06:36:32,NaN,2025-03-07 06:36:32,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False
50138,114743,955,106,42,topic,0,2025-03-07 06:37:49,NaN,2025-03-07 06:37:49,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False
50139,114744,955,104,42,topic,0,2025-03-07 06:37:50,NaN,2025-03-07 06:37:50,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False
50140,114745,955,108,42,topic,0,2025-03-07 06:37:54,NaN,2025-03-07 06:37:54,math,False,True,2025-03-07,0 days 00:00:00,0.000000,False


In [38]:
# Rolling window for recent activity
rolling_window_days = 10

# Convert the 'date' columns to datetime
activity_math['activity_updated'] = pd.to_datetime(activity_math['activity_updated'])
performances_math['time'] = pd.to_datetime(performances_math['time'])

def compute_all_features_for_exam_no_time(exam_row, user_activities, user_exams, window_days=rolling_window_days):

    exam_dt = exam_row['time']

    # Include activities up to and including exam_date
    previous_activities = user_activities[user_activities['activity_updated'] < exam_dt].copy()

    # Rolling window (activities in the last N days, including exam day)
    window_start = exam_dt - pd.Timedelta(days=window_days)
    rolling_activities = previous_activities[previous_activities['activity_updated'] >= window_start].copy()

    features = {}

    # Recent average time per activity (rolling window)
    #total_time_rolling = rolling_activities['time_in_minutes'].sum()
    count_rolling = len(rolling_activities)
    #features['recent_avg_time_per_activity'] = total_time_rolling / count_rolling if count_rolling > 0 else 0

    # Number of days since last activity
    if not previous_activities.empty:
        last_activity_date = previous_activities['activity_updated'].max()
        features['days_since_last_activity'] = (exam_dt - last_activity_date).days
    else:
        features['days_since_last_activity'] = np.nan

    # Total time spent on activities before the exam
    #features['total_time_spent_on_activity_before_exam'] = previous_activities['time_in_minutes'].sum() if not previous_activities.empty else 0

    # Average percentage on past exams
    previous_exams = user_exams[user_exams['time'] < exam_dt]
    features['average_performance_past_exams'] = previous_exams['performance'].mean() if not previous_exams.empty else np.nan

    # Usage Frequency: Average activities per day in rolling window & Active days ratio
    features['avg_activities_per_day_recent'] = count_rolling / window_days if window_days > 0 else np.nan
    if not rolling_activities.empty:
        distinct_days = rolling_activities['activity_updated'].dt.normalize().nunique()
    else:
        distinct_days = 0
    features['active_days_ratio_recent'] = distinct_days / window_days if window_days > 0 else np.nan

    # Activity diversity (rolling window)
    features['diversity_recent'] = rolling_activities['activity_type'].nunique() if not rolling_activities.empty else 0


    return pd.Series(features)

# Loop over each exam (grouped by user) in performances_math and compute all features.
features_list = []

for user_id, user_exams in performances_math.groupby('user_id'):
    # Get corresponding activities for the user from activity_math and sort by date
    user_activities = activity_math[activity_math['user_id'] == user_id].sort_values('activity_updated')
    user_exams_sorted = user_exams.sort_values('time')

    for exam_index, exam_row in user_exams_sorted.iterrows():
        feats = compute_all_features_for_exam_no_time(exam_row, user_activities, user_exams_sorted, rolling_window_days)
        feats['exam_index'] = exam_index
        features_list.append(feats)

# Output df
features_df = pd.DataFrame(features_list).set_index('exam_index')
performances_math_features = performances_math.join(features_df, how='left')

In [22]:
print(performances_math_features.head(5))

      user_id domain test_id  course        date                time  \
9.0         6   math      42    3865  2024-11-23 2024-11-23 10:25:34   
10.0        6   math      48    3865  2025-01-08 2025-01-08 14:48:04   
11.0        6   math      49    3865  2025-01-08 2025-01-08 15:29:07   
12.0        6   math      50    3865  2025-02-04 2025-02-04 15:36:38   
13.0        6   math      54    3865  2024-11-23 2024-11-23 11:26:10   

      percentage  performance  recent_avg_time_per_activity  \
9.0        25.00       -36.04                     19.300000   
10.0       50.00        -1.92                     27.858333   
11.0       66.67        21.23                     27.700000   
12.0       54.55        19.57                      0.000000   
13.0       14.29       -47.71                      8.838095   

      days_since_last_activity  total_time_spent_on_activity_before_exam  \
9.0                        7.0                                 43.016667   
10.0                       0.0      

In [40]:
from sklearn.preprocessing import StandardScaler
# scaling the columns
columns_to_scale = [ 'days_since_last_activity','average_performance_past_exams','avg_activities_per_day_recent','active_days_ratio_recent','diversity_recent']


scaler = StandardScaler()
scaled_values = scaler.fit_transform(performances_math_features[columns_to_scale])
scaled_df = pd.DataFrame(scaled_values, columns=columns_to_scale, index=performances_math_features.index)
remaining_df = performances_math_features.drop(columns=columns_to_scale)
final_df = pd.concat([scaled_df, remaining_df], axis=1)

In [41]:
final_df.dropna(inplace=True)

In [42]:
import statsmodels.formula.api as smf

# Linear Regression Model
mod = smf.ols(
    formula='performance ~  days_since_last_activity + average_performance_past_exams + avg_activities_per_day_recent + active_days_ratio_recent + diversity_recent',
    data=final_df)

# Fit the model
res = mod.fit()

# Print regression results summary
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:            performance   R-squared:                       0.259
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     233.1
Date:                Thu, 24 Apr 2025   Prob (F-statistic):          5.80e-214
Time:                        17:31:37   Log-Likelihood:                -15444.
No. Observations:                3341   AIC:                         3.090e+04
Df Residuals:                    3335   BIC:                         3.094e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept   

If we try to keep all the data we loose two important features regarding the time spent on activity and finish with a model that is less good than the original one (R^2 of 0.259 vs 0.267). DEVELOPP A BIT MORE